In [1]:
import pennylane as qml
import numpy as np
import tensorflow as tf
import cirq

import optim
import torch as torch

2021-11-18 15:27:17.541663: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-18 15:27:17.541694: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Any computational object that can apply quantum operations and return a measurement value is called a quantum device.

In PennyLane, a device could be a hardware device (such as the IBM QX4, via the PennyLane-PQ plugin), or a software simulator (such as Strawberry Fields, via the PennyLane-SF plugin).


In [2]:
dev = qml.device('cirq.simulator', wires=1)

Unitary = arbitrary rotation


In [3]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
##phi = 0
##theta = 0
##omega = 0
angles = [phi,theta,omega]

def real(angles, **kwargs):
    qml.Rot(*angles, wires=0)
        

Generator

In [4]:
def generator(w, **kwargs):
    qml.RZ(w[0], wires=0)
    qml.RY(w[1], wires=0)
    qml.RZ(w[2], wires=0)

Discriminators: DI chooses the input, DO is standard


In [5]:
def DI(w):
    qml.RX(w[0], wires=0)
    qml.RY(w[1], wires=0)
    qml.RZ(w[2], wires=0)

def DO(w):
    qml.RZ(w[0], wires=0)
    qml.RY(w[1], wires=0)
    qml.RZ(w[2], wires=0)


2 nodes:  1 for data-discriminator
          1 for generator-discriminator
          

In [6]:
@qml.qnode(dev, interface="tf", diff_method="parameter-shift")
def real_disc_circuit(angles, disc_weights):
    #DI(disc_weights[9:18])
    real(angles)
    DO(disc_weights)
    return qml.expval(qml.PauliZ(0))

@qml.gradients.param_shift
@qml.qnode(dev)
def real_disc_circuit_gradient(disc_weights):
    #DI(disc_weights[9:18])
    real(angles)
    DO(disc_weights)
    return qml.expval(qml.PauliZ(0))


@qml.qnode(dev, interface="tf", diff_method="parameter-shift")
def gen_disc_circuit(gen_weights, disc_weights):
    #DI(disc_weights[9:18])
    generator(gen_weights)
    DO(disc_weights)
    return qml.expval(qml.PauliZ(0))

@qml.gradients.param_shift
@qml.qnode(dev)
def gen_disc_circuit_gradient_disc(disc_weights):
    #DI(disc_weights[9:18])
    generator(gen_weights)
    DO(disc_weights)
    return qml.expval(qml.PauliZ(0))

@qml.gradients.param_shift
@qml.qnode(dev)
def gen_disc_circuit_gradient_gen(gen_weights):
    #DI(disc_weights[9:18])
    generator(gen_weights)
    DO(disc_weights)
    return qml.expval(qml.PauliZ(0))


Cost Functions

In [7]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(angles, disc_weights)
    
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true


def prob_fake_true(gen_weights, disc_weights):
    
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true


def disc_cost(disc_weights, gen_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost


def gen_cost(disc_weights, gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

Training Initialization:

In [8]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 2) + \
                   np.random.normal(scale=eps, size=(3,))
init_disc_weights = np.random.normal(size=(3,))

gen_weights_grad  = torch.tensor(init_gen_weights, requires_grad=True)
disc_weights_grad = torch.tensor(init_disc_weights, requires_grad=True)

Running training

In [12]:
N = 5     #Number of training cycles
steps = 20 #Number of steps for generator training
ratio = 5  #gen/disc training ratio

for i in range (N):
    
    #training discriminator
    gen_weights_non_grad = torch.tensor(gen_weights_grad.detach().numpy(), requires_grad = False)
    opt = optim.OAdam([disc_weights_grad, gen_weights_non_grad], lr = 0.1)
    
    def closure():
        opt.zero_grad()
        loss = disc_cost(disc_weights_grad, gen_weights_non_grad)
        loss.backward()
        return loss

    for j in range(steps*ratio):
        opt.step(closure)
        if (j+1)%5==0:
            print('step : ',j, ' discriminator cost : ',disc_cost(disc_weights_grad, gen_weights_non_grad).item())
    print('discriminator trained')
    
    #training generator
    disc_weights_non_grad = torch.tensor(disc_weights_grad.detach().numpy(), requires_grad = False)
    opt = optim.OAdam([disc_weights_non_grad, gen_weights_grad], lr = 0.1)

    def closure():
        opt.zero_grad()
        loss = gen_cost(disc_weights_non_grad, gen_weights_grad)
        loss.backward()
        return loss

    for j in range(steps):
        opt.step(closure)
        if (j+1)%5==0:
            print('step : ',j, ' discriminator cost : ',gen_cost(disc_weights_non_grad, gen_weights_grad).item())
        
    print('end of cycle number : ', i+1)


step :  4  discriminator cost :  -0.04048484191298485
step :  9  discriminator cost :  -0.30429197278863285
step :  14  discriminator cost :  -0.47977737337350845
step :  19  discriminator cost :  -0.5457204058766365
step :  24  discriminator cost :  -0.5436265617609024
step :  29  discriminator cost :  -0.5325895249843597
step :  34  discriminator cost :  -0.5352065861225128
step :  39  discriminator cost :  -0.5439396575093269
step :  44  discriminator cost :  -0.5488293915987015
step :  49  discriminator cost :  -0.5490947440266609
step :  54  discriminator cost :  -0.5484528839588165
step :  59  discriminator cost :  -0.5486622378230095
step :  64  discriminator cost :  -0.5491490140557289
step :  69  discriminator cost :  -0.5493121147155762
step :  74  discriminator cost :  -0.5492716431617737
step :  79  discriminator cost :  -0.5492625460028648
step :  84  discriminator cost :  -0.5492959022521973
step :  89  discriminator cost :  -0.5493119359016418
step :  94  discriminator c

In [14]:
print("Prob(real classified as real): ", prob_real_true(disc_weights_grad).item())

Prob(real classified as real):  0.7566307932138443


In [15]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights_grad, disc_weights_grad).item())


Prob(fake classified as real):  0.998358293203637


Comparing output for |0> and printing tthe rotation params of the generator:

In [16]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

bloch_vector_real = qml.map(real, obs, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real([phi, theta, omega])))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights_grad.detach().numpy())))


Real Bloch vector: [0.90096882 0.43388376 0.        ]
Generator Bloch vector: [0.53594226 0.22606558 0.813425  ]


2021-11-11 17:02:32.937693: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-11 17:02:32.937763: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-11 17:02:32.937818: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Hazelnut2): /proc/driver/nvidia/version does not exist
2021-11-11 17:02:32.939148: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
